In [12]:
import os
from dotenv import load_dotenv

load_dotenv()

GOOGLE_API_KEY=os.environ["GOOGLE_API_KEY"] = "your api key"

if GOOGLE_API_KEY:
    print("Gemini API key setup complete.")
else:
    print("Error: GOOGLE_API_KEY not found in environment variables.")

Gemini API key setup complete.


In [13]:
from google.adk.agents import Agent, SequentialAgent, ParallelAgent, LoopAgent
from google.adk.sessions import InMemorySessionService
from google.adk.runners import Runner
from google.adk.models.google_llm import Gemini
from google.adk.runners import InMemoryRunner
from google.adk.tools import AgentTool, FunctionTool, google_search
from google.genai import types

print("✅ ADK components imported successfully.")

✅ ADK components imported successfully.


In [14]:
retry_config=types.HttpRetryOptions(
    attempts=10,  
    exp_base=7,  
    initial_delay=1,
    http_status_codes=[429, 500, 503, 504], 
)


In [15]:

research_agent = Agent(
    name="ResearchAgent",
    model=Gemini(
        model="gemini-2.5-pro",
        retry_options=retry_config
    ),
    instruction="""
    You are a 'Researcher Agent'. Your only purpose is to find information.
    You must use the GoogleSearchTool available to you.
    """,
    tools=[google_search],
    output_key="raw_data",
)

print("✅ research_agent created.")


✅ research_agent created.


In [16]:
reviewer_agent = Agent(
    name="reviewer_agent",
    model=Gemini(
        model="gemini-2.5-pro",
        retry_options=retry_config
    ),
    instruction="""
    You are the 'Reviewer_Agent'. You are an expert academic writer.
    Your job is to write a 'Literature Review' section for a research proposal .
    Read this data properly: {raw_data}
    Write the literature review with the title of Literature review.
    """,
    output_key="literature_review",
)

print("✅ research_agent created.")

✅ research_agent created.


In [17]:
analyst_agent = Agent(
    name="analyst_agent",
    model=Gemini(
        model="gemini-2.5-pro",
        retry_options=retry_config
    ),
    instruction="""
    You are the 'Analyst_Agent'. You are a critical and insightful researcher.
    Your job is to find the "Research Gap".
    Read this properly: {literature_review} and {raw_data}
    analyze literature_review,raw_data and find the research gap.
    """,
    output_key="research_gap",
)

print("✅ research_agent created.")

✅ research_agent created.


In [18]:
methodology_agent = Agent(
    name="methodology_agent",
    model=Gemini(
        model="gemini-2.5-pro",
        retry_options=retry_config
    ),
    instruction="""
    You are the 'Methodology_Agent'. You are a practical and experienced scientist.
    Your job is to propose a research methodology.
    Read this properly: {research_gap}
    analyze research_gap and propose a research methodology.
    """,
    output_key="proposed_methodology",
)

print("✅ research_agent created.")

✅ research_agent created.


In [19]:
aggregator_agent = Agent(
    name="AggregatorAgent",
    model=Gemini(
        model="gemini-2.5-pro",
        retry_options=retry_config
    ),

    instruction="""
    You are the 'Aggregator_Agent'. You are an expert academic writer.
    Your job is to compile a research proposal.
    
    research proposal should include the following sections:
    section 1 
    Literature Review:
    {literature_review}
    section 2
    Research Gap:
    {research_gap}
    section 3
    Methodology:
    {proposed_methodology}
    
    Write a cohesive research proposal that seamlessly integrates the literature review, research gap, and proposed methodology.""",
    tools=[AgentTool(research_agent),AgentTool(reviewer_agent),AgentTool(analyst_agent),AgentTool(methodology_agent)],
    output_key="Final_Result",
)

print("✅ aggregator_agent created.")

✅ aggregator_agent created.


In [ ]:

APP_NAME = "Research Proposal Assistant"  
USER_ID = "01"  
SESSION = "sid001"  

MODEL_NAME = "gemini-2.5-pro"
root_agent = SequentialAgent(
    name="ResearchCoordinator",
    sub_agents=[research_agent, reviewer_agent, analyst_agent,methodology_agent, aggregator_agent],
)
print("✅ Coordinator_agent.")
session_service = InMemorySessionService()


runner = Runner(agent=root_agent, app_name=APP_NAME, session_service=session_service)

print("✅ Stateful agent initialized!")
print(f"   - Application: {APP_NAME}")
print(f"   - User: {USER_ID}")
print(f"   - Using: {session_service.__class__.__name__}")



✅ Coordinator_agent.
✅ Stateful agent initialized!
   - Application: Research Proposal Assistant
   - User: 01
   - Using: InMemorySessionService


In [ ]:
async def run_session(
    runner_instance: Runner,
    user_queries: list[str] | str = None,
    session_name: str = "default",
):
    print(f"\n ### Session: {session_name}")

    
    app_name = runner_instance.app_name

    
    try:
        session = await session_service.create_session(
            app_name=app_name, user_id=USER_ID, session_id=session_name
        )
    except:
        session = await session_service.get_session(
            app_name=app_name, user_id=USER_ID, session_id=session_name
        )

    
    if user_queries:
        
        if type(user_queries) == str:
            user_queries = [user_queries]

        
        for query in user_queries:
            print(f"\nUser > {query}")

            
            query = types.Content(role="user", parts=[types.Part(text=query)])

            
            async for event in runner_instance.run_async(
                user_id=USER_ID, session_id=session.id, new_message=query
            ):
                
                if event.content and event.content.parts:
                    
                    if (
                        event.content.parts[0].text != "None"
                        and event.content.parts[0].text
                    ):
                        print(f"AI > ", event.content.parts[0].text)
    else:
        print("No queries!")


print("✅ Helper functions defined.")

✅ Helper functions defined.


In [22]:
await run_session(
    runner,
    [
        "Feature Extraction of Rice Varieties for Morphological and Visual Similarity Analysis using Deep Learning?",
        "Do you remember the previous topic we talked about?",  
    ],
    "sid001",
)


 ### Session: sid001

User > Feature Extraction of Rice Varieties for Morphological and Visual Similarity Analysis using Deep Learning?
AI >  ### Deep Learning Revolutionizes Rice Variety Analysis Through Automated Feature Extraction

Deep learning models, particularly Convolutional Neural Networks (CNNs), are at the forefront of a technological revolution in agriculture, specifically in the morphological and visual similarity analysis of rice varieties. These advanced neural networks automatically extract key features from images of rice grains, enabling highly accurate classification and analysis that surpasses traditional, manual methods in both speed and objectivity.

The core strength of deep learning in this field lies in its ability to automatically learn and identify discriminating features directly from image data. This eliminates the need for manual feature extraction, a process that is not only time-consuming but also prone to human error and subjectivity. CNNs can identify

AI >  ### **Research Proposal**

**Research Title:** A Framework for Robust, Scalable, and Quantitative Morphological Similarity Analysis of Rice Varieties using Deep Metric Learning and Explainable AI.

### **1. Literature Review**

The classification and analysis of rice varieties are critical for agricultural quality control, trade regulation, and genetic improvement. Traditionally, these tasks have relied on manual inspection by trained experts, a process that is inherently subjective, labor-intensive, and susceptible to human error. In recent years, the field has witnessed a paradigm shift towards automated, image-based analysis, with deep learning emerging as a transformative technology for extracting morphological and visual features from rice grains.

The primary advantage of deep learning, particularly Convolutional Neural Networks (CNNs), lies in its capacity for automated feature extraction. Unlike conventional machine learning approaches that require manual engineering of f

AI >  Of course. I remember our conversation in detail.

We discussed and formulated a comprehensive research proposal titled: **"A Framework for Robust, Scalable, and Quantitative Morphological Similarity Analysis of Rice Varieties using Deep Metric Learning and Explainable AI."**

The proposal was built upon a logical progression:

1.  **Literature Review:** We first established that current research successfully uses deep learning (CNNs) for classifying rice varieties with high accuracy, but only in controlled environments with a very limited number of classes (e.g., five).

2.  **Identified Research Gaps:** We then critically analyzed this state-of-the-art to identify four key weaknesses:
    *   **Lack of Scalability:** The methods are not proven to work on the hundreds of varieties that exist in the real world.
    *   **Lack of Robustness:** The models are not tested against real-world image conditions, such as impurities, damaged grains, or piles of overlapping grains.
    *   

In [ ]:
import json
import datetime
import os

async def save_full_session_log(session_service, app_name, user_id, session_id):

    
    try:
        newsession = await session_service.get_session(
            app_name=app_name, user_id=user_id, session_id=session_id
        )
    except Exception as e:
        print(f" session not found: {e}")
        return

    
    timestamp = datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
    log_data = {
        "session_meta": {
            "app_name": app_name,
            "session_id": session_id,
            "saved_at": timestamp
        },
        "conversation_history": []
    }

    
    
    if hasattr(newsession, 'events'):
        for msg in newsession.message:
            
            content_text = ""
            if msg.content and msg.content.parts:
                parts_text = []
                for p in msg.content.parts:
                    if hasattr(p, 'text') and p.text:
                        parts_text.append(p.text)
                content_text = " ".join(parts_text)
            
            
            entry = {
                "role": msg.role,  
                "content": content_text
            }
            log_data["conversation_history"].append(entry)


    filename = f"Research_Log_{session_id}_{timestamp}.json"
    
    try:
        with open(filename, "w", encoding="utf-8") as f:
            json.dump(log_data, f, indent=4, ensure_ascii=False)
        print(f"✅ Observability Log saved succesfully: {filename}")
        print(f"📂 fille path: {os.path.abspath(filename)}")
    except Exception as e:
        print(f"problem to save log: {e}")

print("✅ Logging function ready.")

✅ Logging function ready.


In [ ]:
async def save_log_manual(session_name: str):
    print(f"\n--- 💾 Saving Logs for: {session_name} ---")
    
    app_name = APP_NAME 
    user_id = USER_ID   
    
    timestamp = datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
    
    
    try:
        fresh_session = await session_service.get_session(
            app_name=app_name, 
            user_id=user_id, 
            session_id=session_name
        )
        messages = fresh_session.events
        print(f"DEBUG: Found {len(messages)} messages to save.")
    except Exception as e:
        print(f" Error fetching session history: {e}")
        return


    log_data = {
        "session_meta": {
            "app_name": app_name,
            "session_id": session_name,
            "saved_at": timestamp,
            "total_messages": len(messages)
        },
        "conversation_history": []
    }

    for msg in messages:
        content_text = ""
        if msg.content and msg.content.parts:
            parts_text = []
            for p in msg.content.parts:
                if hasattr(p, 'text') and p.text:
                    parts_text.append(p.text)
            content_text = " ".join(parts_text)
        
        log_data["conversation_history"].append({
            "role": msg.role,
            "content": content_text
        })


    filename = f"Research_Log_{session_name}_{timestamp}.json"
    try:
        with open(filename, "w", encoding="utf-8") as f:
            json.dump(log_data, f, indent=4, ensure_ascii=False)
        print(f" Log Saved Successfully: {filename}")
    except Exception as e:
        print(f" Failed to save file: {e}")

print("save_log_manual created")

✅ Two separate functions defined: 'run_session_chat' and 'save_log_manual'


In [48]:

await save_full_session_log(
    session_service=session_service,
    app_name=APP_NAME,
    user_id=USER_ID,
    session_id=SESSION
)

AttributeError: 'Event' object has no attribute 'role'